In [85]:
from bs4 import BeautifulSoup
import requests
import json


import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
import config

import pprint

import random 
from time import sleep

import pandas as pd

In [86]:
Client_ID = config.client_id
Client_Secret = config.client_secret

sp = spotipy.Spotify(auth_manager = SpotifyClientCredentials(client_id = Client_ID, client_secret = Client_Secret))

# 100 Hot songs (Scraping the Billbord Website) 
### Scraping for 2021

In [248]:
#------scrapping 2021 data-------#

url = "https://www.billboard.com/charts/hot-100"
response = requests.get(url)
newest_soup = BeautifulSoup(response.content, 'html.parser')

In [249]:
response.status_code

200

In [250]:
#--------2021 song names--------#

newest_song = newest_soup.select('span.chart-element__information__song.text--truncate.color--primary')
newest_song = [t.text for t in newest_song]


#--------2021 artist names--------#

newest_artist = newest_soup.select('span.chart-element__information__artist.text--truncate.color--secondary')
newest_artist = [t.text for t in newest_artist]

In [166]:
#------scrapping 2019,2020 data-------#

str = 'https://www.billboard.com/charts/year-end/{}/hot-100-songs'
years = [ 2006, 2007 ,2008, 2009, 2010,2011, 2012, 2013, 2014, 2015, 2016, 2017, 2018 ,2019, 2020]

song_pool_soups = []
artist_pool_soups = []

for year in years:
    url = str.format(year)
    response = requests.get(url)
    #print(response.status_code)
    
    soup = BeautifulSoup(response.content)
    song_pool_soups.append(soup.find_all('div', class_ = 'ye-chart-item__title'))
    artist_pool_soups.append(soup.find_all('div', class_ = 'ye-chart-item__artist'))
    
    
    wait_time = random.randint(1,4)
    sleep(wait_time)

In [167]:
#------songs & artists 2006-2020------#

for i in range(0,len(years)):
    song_pool_soups[i] = [t.text.replace('\n','') for t in song_pool_soups[i]]
    artist_pool_soups[i] = [t.text.replace('\n','') for t in artist_pool_soups[i]]    

In [ ]:
#release_year = soup.find('span', class_ ='dropdown__year-select-date').get_text()
#song_release_year = [2006]*100 + [2007]*100 + [2008]*100 + [2009]*100 + [2010]*100 + [2011]*100 + [2012]*100 + [2013]*100 + [2014]*100 + [2015]*100 + [2016]*100 + [2017]*100 + [2018]*100 + [2019]*100 + [2020]*100 + [2021]*100

In [168]:
#----------concatinate the 2006-2020 years data together----------#

song_name = []
song_artist = []

for i in range(0,len(older_song_soups)):
    song_name += older_song_soups[i]
    song_artist += older_artist_soups[i]

In [256]:
#-----current hot topic dataframe 2021----------#

df_current = pd.DataFrame({'song_name': newest_song, 'artist': newest_artist})
df_current.head(5)

,song_name,artist
0,Way 2 Sexy,Drake Featuring Future & Young Thug
1,Girls Want Girls,Drake Featuring Lil Baby
2,Fair Trade,Drake Featuring Travis Scott
3,Champagne Poetry,Drake
4,Knife Talk,Drake Featuring 21 Savage & Project Pat


In [292]:
#----- hot topic dataframe 2006-2020----------#

df = pd.DataFrame({'song_name': song_name, 'artist': song_artist})
df.head(20)

,song_name,artist
0,Bad Day,Daniel Powter
1,Temperature,Sean Paul
2,Promiscuous,Nelly Furtado Featuring Timbaland
3,You're Beautiful,James Blunt
4,Hips Don't Lie,Shakira Featuring Wyclef Jean
5,Unwritten,Natasha Bedingfield
6,Crazy,Gnarls Barkley
7,Ridin',Chamillionaire Featuring Krayzie Bone
8,SexyBack,Justin Timberlake
9,Check On It,Beyonce Featuring Slim Thug


In [257]:
#---------Making the dataframe content lowercase-----------#

df_current["song_name"] = df_current["song_name"].apply(lambda x: x.lower())
df_current["artist"] = df_current["artist"].apply(lambda x: x.lower())

In [258]:
#--------maching sequence of input with hot songs------------#

from difflib import SequenceMatcher 

similarity = []
def similar(a):
    for i in range (0,len(newest_song)):
        similarity.append(SequenceMatcher(None, a, newest_song[i]).ratio())
        most_similar = similarity.index(max(similarity))
    return newest_song[most_similar]           

In [259]:
#--------Taking input ------------#

def user_song():
    input_song = input('Enter Your Favorite Song: ')
    
    similarity = []
    if input_song.lower() in df_current.song_name.values:
        print('The recommended song for you is: ',random.choice(df_current['song_name']),'\n')
            
    elif similar(input_song.lower()) in df_current.song_name.values:
        print('Do you mean: ', similar(input_song.lower()))
        user_song()
    
    else:
        print('Sorry, Pick another song\n')
        user_song()

In [ ]:
user_song()

In [242]:
#----------extracting playlists from spotify API---------#


cats = [cat['id'] for cat in sp.categories()['categories']['items']]


In [247]:
cats.pop(8)

'alternative'

In [248]:
#-----------determining the id of the playlists --------#

playlist_ids = []
cats = [cat['id'] for cat in sp.categories()['categories']['items']]

for i in range(0,len(cats)):
    playlist_ids.append(sp.category_playlists(cats[i])['playlists']['items'][0]['id'])
    
    wait_time = random.randint(1,3)
    sleep(wait_time)

In [249]:
playlist_ids

['37i9dQZF1DXcBWIGoYBM5M',
 '37i9dQZF1DWTLSN7iG21yC',
 '37i9dQZF1DXcBWIGoYBM5M',
 '37i9dQZF1DWU8quswnFt3c',
 '37i9dQZF1DX3rxVfibe1L0',
 '37i9dQZF1DX4UtSsGT1Sbe',
 '37i9dQZF1DX0XUsuxWHRQd',
 '37i9dQZF1DX0BxHamIEkKV',
 '37i9dQZF1DWVgsJtp58d1t',
 '37i9dQZF1DWTyiBJ6yEqeu',
 '37i9dQZF1DX9uKNf5jGX6m',
 '37i9dQZF1DX76Wlfdnj7AP',
 '37i9dQZF1DX4WYpdgoIcn6',
 '37i9dQZF1DX4sWSpwq3LiO',
 '37i9dQZF1DXdbkmlag2h7b',
 '37i9dQZF1DXaXB8fQg7xif',
 '37i9dQZF1DXdTCdwCKzXwo',
 '37i9dQZF1DWTcqUzwhNmKv',
 '37i9dQZF1DWViUlcvfltyZ',
 '37i9dQZF1DX2ENAPP1Tyed']

In [251]:
len(playlist_ids)

20

In [274]:
len(get_playlist_tracks('breezz', playlist_ids[5]))

150

In [31]:
def get_playlist_tracks(username ,playlist_id):
    results = sp.user_playlist_tracks(username,playlist_id)
    tracks = results['items']
    while results['next']:
        results = sp.next(results)
        tracks.extend(results['items'])
    return tracks

In [276]:
#-----------determining the tracks'name, artists'name and uri for all the tracks in the playlists --------#

track_names = []                      
track_artist = []
track_uri = []                           

for i in range(5,6):
    tracks = get_playlist_tracks('breezz', playlist_ids[i])
    
    for j in range(0,len(tracks)):
        track_names.append(tracks[j]['track']['name'])
        track_uri.append(tracks[j]['track']['uri'])
        
        artist = tracks[j]['track']['artists']
        artists = []
        
        for k in range(0, len(artist)):
            artists.append(tracks[j]['track']['artists'][k]['name']) 
        
        track_artist.append(" & ".join(artists)) 
            
        wait_time = random.randint(1,5)
        sleep(wait_time)            

In [34]:
tracks = get_playlist_tracks('breezz', playlist_ids[0])
len(tracks)

150

In [48]:
track_names

['Hold On',
 'Wonder',
 'Beautiful Mistakes (feat. Megan Thee Stallion)',
 'Glad You Exist',
 'Our Song',
 'Leave Before You Love Me (with Jonas Brothers)',
 'imagine',
 'dancing in the kitchen',
 'if we never met (feat. Kelsea Ballerini)',
 'Butterflies',
 'What Other People Say',
 'Summer of Love (Shawn Mendes & Tainy)',
 'working',
 'Wave of You',
 'deja vu',
 'I Should Probably Go To Bed',
 "We're Good",
 'WITHOUT YOU (with Miley Cyrus)',
 'Shivers',
 'Remember This',
 'Hold On',
 'Wonder',
 'Beautiful Mistakes (feat. Megan Thee Stallion)',
 'Glad You Exist',
 'Our Song',
 'Leave Before You Love Me (with Jonas Brothers)',
 'imagine',
 'dancing in the kitchen',
 'if we never met (feat. Kelsea Ballerini)',
 'Butterflies',
 'What Other People Say',
 'Summer of Love (Shawn Mendes & Tainy)',
 'working']

In [277]:
#--------audio features---------#
list_of_audio_feature_dict = []

for i in range(0,len(track_uri)):
    audio_feature_dict = sp.audio_features(track_uri[i])[0]
    list_of_audio_feature_dict.append(audio_feature_dict)
    
    wait_time = random.randint(1,3)
    sleep(wait_time) 


In [169]:
len(list_of_audio_feature_dict)

0

[{'danceability': 0.605,
  'energy': 0.377,
  'key': 11,
  'loudness': -6.213,
  'mode': 1,
  'speechiness': 0.0443,
  'acousticness': 0.733,
  'instrumentalness': 0,
  'liveness': 0.0808,
  'valence': 0.519,
  'tempo': 73.877,
  'type': 'audio_features',
  'id': '3H3r2nKWa3Yk5gt8xgmsEt',
  'uri': 'spotify:track:3H3r2nKWa3Yk5gt8xgmsEt',
  'track_href': 'https://api.spotify.com/v1/tracks/3H3r2nKWa3Yk5gt8xgmsEt',
  'analysis_url': 'https://api.spotify.com/v1/audio-analysis/3H3r2nKWa3Yk5gt8xgmsEt',
  'duration_ms': 194853,
  'time_signature': 4},
 {'danceability': 0.556,
  'energy': 0.661,
  'key': 6,
  'loudness': -5.052,
  'mode': 0,
  'speechiness': 0.204,
  'acousticness': 0.3,
  'instrumentalness': 0,
  'liveness': 0.101,
  'valence': 0.668,
  'tempo': 168.56,
  'type': 'audio_features',
  'id': '6PERP62TejQjgHu81OHxgM',
  'uri': 'spotify:track:6PERP62TejQjgHu81OHxgM',
  'track_href': 'https://api.spotify.com/v1/tracks/6PERP62TejQjgHu81OHxgM',
  'analysis_url': 'https://api.spotify.c

In [278]:
df = pd.DataFrame(list_of_audio_feature_dict)

In [279]:
df['artist'] = track_artist

In [280]:
df['song_name'] = track_names

In [281]:
df.to_csv('audio_features_album9.csv', index = False)

In [282]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 150 entries, 0 to 149
Data columns (total 20 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   danceability      150 non-null    float64
 1   energy            150 non-null    float64
 2   key               150 non-null    int64  
 3   loudness          150 non-null    float64
 4   mode              150 non-null    int64  
 5   speechiness       150 non-null    float64
 6   acousticness      150 non-null    float64
 7   instrumentalness  150 non-null    float64
 8   liveness          150 non-null    float64
 9   valence           150 non-null    float64
 10  tempo             150 non-null    float64
 11  type              150 non-null    object 
 12  id                150 non-null    object 
 13  uri               150 non-null    object 
 14  track_href        150 non-null    object 
 15  analysis_url      150 non-null    object 
 16  duration_ms       150 non-null    int64  
 1